# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [ ]:
!pip install mlflow
!pip install torch==1.8.1
!pip install pytorch-lightning
!pip install transformers
!pip install cloudpathlib

In [ ]:
import sys
sys.path.append('../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [ ]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import mlflow
import mlflow.sagemaker
import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

In [ ]:
from deep.constants import *
from deep.utils import *

In [ ]:
%load_ext autoreload
%autoreload 2

## Data

Select which MLFlow model to deploy

In [ ]:
logged_model = 's3://deep-mlflow-artifact/2/f3b4e0f9a0364f8dbfe1563b248348a1/artifacts/model'

## Sagemaker Prep

SageMaker configuration

In [ ]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
prefix = "pl/example"

We use the MLFlow SageMaker API. The parameters you may want to change are:

- `pl-example`, choose the name you want for your API
- `instance-type`, select the instance type you want

In [ ]:
mlflow.sagemaker.deploy(
    'pl-example',
    logged_model,
    execution_role_arn=SAGEMAKER_ROLE_ARN,
    image_url='961104659532.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:latest',
    region_name='us-east-1',
    instance_type='ml.p2.xlarge',
    mode='replace'
)